In [9]:
from pyteaser import Summarize, SummarizeUrl
from pprint import pprint
import pandas as pd
import numpy as np
import sqlite3
from sqlite3 import Error
from datetime import datetime
import requests
import urllib

In [10]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print("DB Connection setup")
    except Error as e:
        print(e)
    finally:
        if conn:
            return conn
        else:
            print("DB Connection failed")


if __name__ == '__main__':
    conn = create_connection("./db/news.db")

DB Connection setup


In [11]:
def getSiteByName(name):
    cur = conn.cursor()
    cur.execute("SELECT * FROM site where name=?",(name,))
    rows = cur.fetchall()
    return rows

In [12]:
site = getSiteByName("BBC News")
site_url = site[0][2]
site_id = site[0][0]

In [13]:
# BBC top news
def NewsFromBBC():
     
    # BBC news api
    main_url = site_url
 
    # fetching data in json format
    open_bbc_page = requests.get(main_url).json()
 
    # getting all articles in a string article
    article = open_bbc_page["articles"]
 
    # empty list which will 
    # contain all trending news
    results = []
    url = []
    Img_link = []
     
    for ar in article:
        results.append(ar["title"])
        url.append(ar["url"])
        Img_link.append(ar["urlToImage"])
         
        number_BBC = len(results)
    
    return results, url, number_BBC, Img_link
 

In [18]:
if __name__ == '__main__':
     
# function call

    results, url, number_BBC, Img_link = NewsFromBBC()
    
    df_BBC = pd.DataFrame(index=range(0,number_BBC-1), columns = ['Sno', 'Title', 'URL','Summary', 'Img_URL'])


    for i in range(1, number_BBC):

        summaries = SummarizeUrl(url[i])
        
        df_BBC.iloc[i-1, 0] = i
        df_BBC.iloc[i-1, 1] = results[i]
        df_BBC.iloc[i-1, 2] = url[i]
        df_BBC.iloc[i-1, 3] = summaries
        df_BBC.iloc[i-1, 4] = Img_link[i]


In [19]:
def remove_str(summm_l, SS):
    summm = ''.join(summm_l)
    summm = summm.encode('utf-8').decode('utf-8', 'ignore')
    if SS in summm:
        summm = summm.replace(SS, '')
    
    return summm

for i in range(0, df_bb.shape[0]):
    df_BBC['Summary'][i] = remove_str(df_BBC['Summary'][i], "u'")

NameError: name 'df_bb' is not defined

In [9]:
def insertNews(conn,news):    
    sql = ''' INSERT INTO news(site_id,create_date,title,url,summary,img_url,latest)
              VALUES(?,?,?,?,?,?,1) '''
    cur = conn.cursor()
    cur.execute(sql, news)
    return cur.lastrowid

def markNews(conn, site_id):
    cur = conn.cursor()
    cur.execute("UPDATE news set latest=0 where latest=1 and site_id=? ",(site_id,))

In [10]:
now = datetime.now()
# dd/mm/YY
dt_string = now.strftime("%d/%m/%Y")
markNews(conn, site_id)
for index, row in df_BBC.iterrows():
    title = row['Title']
    url = row['URL']
    summary = str(row['Summary'])
    img_url = row['Img_URL']
    news = (site_id,dt_string,title,url,summary,img_url)
    news_id = insertNews(conn,news)
    print("News %d has been created" % news_id)
    
conn.commit()

News 106 has been created
News 107 has been created
News 108 has been created
News 109 has been created
News 110 has been created
News 111 has been created
News 112 has been created
News 113 has been created
News 114 has been created
